In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

import MathOptInterface as MOI
import Ipopt 
import ForwardDiff as FD 
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
import MeshCat as mc 
using Printf

include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","quadrotor.jl"))

  Activating project at `~/Research/code/DroneTransportation/notebook`


animate_quadrotor (generic function with 1 method)

# Problem Setting

## Cost

In [2]:
function task_cost(params::NamedTuple, Z::Vector)::Real
    # compute the cost 
    xend = Z[params.idx.x[end]]
    J = (xend - params.xf)'*params.Q*(xend - params.xf)
    for k = 1:(params.N-1)
        x = Z[params.idx.x[k]]
        u = Z[params.idx.u[k]]
        J += (x - params.xf)'*params.Q*(x - params.xf) + u'*params.R*u
    end
    return J
end

task_cost (generic function with 1 method)

## Equality constraint

In [3]:
function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
    dynamics = combined_dynamics
    x_mid =
        0.5 * (x1 + x2) +
        0.125 * dt * (dynamics(params.model, x1, u) - dynamics(params.model, x2, u))
    return x1 +
        1 / 6 *
        dt *
        (
            dynamics(params.model, x1, u) +
            4 * dynamics(params.model, x_mid, u) +
            dynamics(params.model, x2, u)
        ) - x2
end

function eq_constraints(params::NamedTuple, Z::Vector)::Vector
    c = zeros(eltype(Z), 15*(params.N)+12)
    
    # dynamic constrains
    for i = 1:(params.N-1)
        xi = Z[params.idx.x[i]]
        xip1 = Z[params.idx.x[i+1]]
        ui = Z[params.idx.u[i]]        
        # dynamics constraints
        c[12*(i+1) .+ (1:12)] = hermite_simpson(params, xi, xip1, ui, params.model.dt)
        # rope force constrains
        ui_lift = ui[1:7]
        ui_load = ui[8:10]
        c[12*(i+1) .+ (13:15)] = ui_lift[5:7] - ui_load
    end

    # initial condition
    x1 = Z[params.idx.x[1]]
    c[1:12] = x1 - params.xi

    # final condition
    xf = Z[params.idx.x[end]]
    c[13:24] = xf - params.xf
    
    return c
end

eq_constraints (generic function with 1 method)

## Inequality constraint

In [4]:
function ineq_constraints(params::NamedTuple, Z::Vector)::Vector
    c = zeros(eltype(Z), 4*(params.N-1))
    for i = 1:(params.N-1)
        u = Z[params.idx.u[i]]
        c[4*(i-1) .+ (1:4)] = u
    end
    return c
end

ineq_constraints (generic function with 1 method)

## Task

In [5]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    
    # our Z vector is [xi, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

"""
    quadrotor navigation

Function for returning collision free trajectories for 3 quadrotors. 

Outputs:
    x::Vector{Vector}  # state trajectory for quad
    u::Vector{Vector}  # control trajectory for quad
    t_vec::Vector
    params::NamedTuple

The resulting trajectories should have dt=0.2, tf = 5.0, N = 26
where all the x's are length 26, and the u's are length 25. 

"""
function quadrotor_navigation(;verbose=true)
    
    # problem size 
    nx_lift = 12
    nx_load = 6
    nx = 12 + 6
    nu_lift = 7
    nu_load = 3
    nu = 7 + 3
    dt = 0.05
    tf = 5.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial conditions and goal states 
    xi = zeros(12)
    xf = zeros(12)
    xf[1] = 5.0
    
    # load all useful things into params 
    Q = 1.0 * diagm([1.0 * ones(3); 0.1 * ones(3); 1.0 * ones(3); 1.0 * ones(3)])
    R = 0.1 * I(nu)
    model = (mass=0.5,
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0,0,-9.81],
        L=0.1750, # drone arm length
        kf=1.0, u_max = 10.0/4.0, 
        km=0.0245,dt = dt)
    params = (
        N = N, 
        nx = nx, 
        nx_lift = nx_lift,
        nx_load = nx_load,
        nu = nu, 
        nu_lift = nu_lift,
        nu_load = nu_load,
        Q = Q, 
        R = R, 
        model = model, 
        xi = xi,
        xf = xf,
        idx = idx
    )
    
    x_guess = range(params.xi, params.xf, length=params.N)
    z0 = zeros(params.idx.nz)
    for i = 1:params.N
        z0[params.idx.x[i][1]] = x_guess[i][1]
    end

    diff_type = :auto

    Z = fmincon(
        task_cost, 
        eq_constraints,
        ineq_constraints, 
        ones(params.idx.nz) .* -10,
        ones(params.idx.nz) .* 10,
        ones(4*(N-1)) .* 0.0, 
        ones(4*(N-1)) .* params.model.u_max,
        z0, 
        params,
        diff_type; 
        tol = 1e-6,
        c_tol = 1e-6,
        max_iters=10_000, 
        verbose = true
    )
    
    # return the trajectories 
    xs = [zeros(12)  for _ = 1:params.N]
    us = [zeros(4)  for _ = 1:(params.N-1)]
    for i = 1:params.N
        x = Z[params.idx.x[i]]
        xs[i] = x[1:12]
        if i < params.N
            us = Z[params.idx.u[i]]
        end
    end
        
    return xs, us, t_vec, params 
end

quadrotor_navigation

In [6]:
xs, us, t_vec, params = quadrotor_navigation()
display(animate_quadrotor(xs, xs, params.model.dt))

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------


---------testing constraint Jacobian----------------


In [ ]:
xs = hcat(xs...)
us = hcat(us...)
plot(t_vec, xs[1:3], title = "Position", label = ["x" "y" "z"], xlabel = "Time [s]", ylabel = "Position [m]")
display(plot!(t_vec, xs[4:6], title = "Velocity", label = ["vx" "vy" "vz"], xlabel = "Time [s]", ylabel = "Velocity [m/s]"))